# Snowflake Cost Analysis Agent

## Overview
This notebook creates a Cortex Analyst agent that answers natural language questions about your Snowflake costs.

## What This Notebook Does
- Creates a dedicated warehouse to track agent costs
- Creates 3 optimized views for cost analysis
- Generates a semantic model for natural language queries
- Provides instructions to create the Cortex Analyst agent

## Key Features
- ✅ **Flexible time periods** - Ask about any time range (up to 365 days)
- ✅ **Product categories** - Analyze costs by AI, Storage, Compute, etc.
- ✅ **Natural language** - Use plain English to query your data
- ✅ **Cost tracking** - Monitor the agent's own costs

## Example Questions You Can Ask
- What is my daily AI cost for the last 30 days?
- Show me costs for warehouse X in March 2025
- Compare my costs from last week to this week
- Show me Q1 2025 costs by product categories
- What are my top 5 most expensive warehouses?
- How much did I spend on compute vs storage in January?
- How much does the cost agent itself cost?

## Step 1: Configuration

Update the cost per credit value below to match your Snowflake contract.

This is the **only variable** you need to configure!

In [ ]:
-- Configuration
USE ROLE ACCOUNTADMIN;

-- Update this value to match your Snowflake cost per credit
SET COST_PER_CREDIT = 2.5;  -- Your Snowflake cost per credit in USD

-- Create dedicated warehouse for the cost agent
-- This allows you to track exactly how much the agent itself costs
CREATE WAREHOUSE IF NOT EXISTS COST_AGENT_WH
    WAREHOUSE_SIZE = 'XSMALL'
    AUTO_SUSPEND = 60              -- Suspends after 1 minute of inactivity
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = FALSE
    COMMENT = 'Dedicated warehouse for Cost Analysis Agent';

USE WAREHOUSE COST_AGENT_WH;

-- Create database and schema for cost views
CREATE DATABASE IF NOT EXISTS COST_ANALYSIS_DB;
CREATE SCHEMA IF NOT EXISTS COST_ANALYSIS_DB.COST_VIEWS;

USE DATABASE COST_ANALYSIS_DB;
USE SCHEMA COST_VIEWS;

-- Verify configuration
SELECT 
    'Configuration complete!' AS STATUS,
    'Cost per credit: $' || $COST_PER_CREDIT AS COST_INFO,
    'Warehouse: COST_AGENT_WH' AS WAREHOUSE_INFO;

## Step 2: Create Cost Analysis Views

This step creates 3 views that provide cost insights:

1. **COST_WITH_CATEGORIES** - Service costs grouped by product category (AI, Storage, Compute, etc.)
2. **WAREHOUSE_COSTS** - Warehouse-level costs with compute and cloud services breakdown
3. **QUERY_COSTS** - Query-level cost estimates by user and database

**Important**: These views do NOT filter by time. They expose all available historical data (up to 365 days).
When you ask questions, the Cortex Analyst will automatically filter by your requested time period.

**Why use views?**
- Simplify complex cost calculations
- Add business-friendly product categories
- Convert credits to USD automatically
- Improve query performance

In [ ]:
-- View 1: Cost with Product Categories
-- Groups Snowflake services into business-friendly categories

CREATE OR REPLACE VIEW COST_WITH_CATEGORIES AS
SELECT 
    USAGE_DATE,
    SERVICE_TYPE,
    CASE 
        -- AI and ML Services
        WHEN SERVICE_TYPE IN ('SNOWPARK', 'SNOWPARK_CONTAINER_SERVICES', 'CORTEX_SEARCH', 
                              'CORTEX_ANALYST', 'CORTEX_FINE_TUNING', 'CORTEX_INFERENCE',
                              'ML_FUNCTIONS', 'SNOWFLAKE_ML') 
            THEN 'AI & Machine Learning'
        -- Data Transformation & Processing
        WHEN SERVICE_TYPE IN ('WAREHOUSE_METERING', 'COMPUTE', 'QUERY_ACCELERATION',
                              'MATERIALIZED_VIEW', 'PIPE', 'TASK', 'AUTOMATIC_CLUSTERING') 
            THEN 'Data Transformation & Compute'
        -- Data Storage
        WHEN SERVICE_TYPE IN ('STORAGE', 'DATA_TRANSFER', 'DATABASE_STORAGE', 
                              'STAGE_STORAGE', 'FAILSAFE_STORAGE') 
            THEN 'Data Storage'
        -- Data Sharing & Collaboration
        WHEN SERVICE_TYPE IN ('DATA_SHARING', 'REPLICATION', 'EXTERNAL_FUNCTIONS',
                              'LISTING_AUTO_FULFILLMENT') 
            THEN 'Data Sharing & Collaboration'
        -- Cloud Services
        WHEN SERVICE_TYPE IN ('CLOUD_SERVICES', 'CLOUD_SERVICES_ONLY') 
            THEN 'Cloud Services'
        -- Serverless Features
        WHEN SERVICE_TYPE IN ('SERVERLESS_TASK', 'SERVERLESS_FEATURE') 
            THEN 'Serverless Features'
        -- Search & Optimization
        WHEN SERVICE_TYPE IN ('SEARCH_OPTIMIZATION', 'AUTOMATIC_RECLUSTERING') 
            THEN 'Search & Optimization'
        ELSE 'Other Services'
    END AS PRODUCT_CATEGORY,
    CREDITS_USED,
    CREDITS_USED * $COST_PER_CREDIT AS COST_USD,
    ACCOUNT_NAME
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY;

SELECT 'View COST_WITH_CATEGORIES created successfully' AS STATUS;

In [ ]:
-- View 2: Warehouse Costs
-- Provides warehouse-level costs with compute vs cloud services breakdown

CREATE OR REPLACE VIEW WAREHOUSE_COSTS AS
SELECT
    START_TIME,
    END_TIME,
    WAREHOUSE_NAME,
    CREDITS_USED,
    CREDITS_USED_COMPUTE,
    CREDITS_USED_CLOUD_SERVICES,
    CREDITS_USED * $COST_PER_CREDIT AS COST_USD,
    CREDITS_USED_COMPUTE * $COST_PER_CREDIT AS COMPUTE_COST_USD,
    CREDITS_USED_CLOUD_SERVICES * $COST_PER_CREDIT AS CLOUD_SERVICES_COST_USD,
    WAREHOUSE_ID
FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY;

SELECT 'View WAREHOUSE_COSTS created successfully' AS STATUS;

In [ ]:
-- View 3: Query Costs
-- Estimates query-level costs by proportionally allocating warehouse costs

CREATE OR REPLACE VIEW QUERY_COSTS AS
WITH warehouse_hourly_credits AS (
    SELECT 
        DATE_TRUNC('HOUR', START_TIME) AS HOUR,
        WAREHOUSE_NAME,
        SUM(CREDITS_USED) AS TOTAL_CREDITS
    FROM SNOWFLAKE.ACCOUNT_USAGE.WAREHOUSE_METERING_HISTORY
    GROUP BY DATE_TRUNC('HOUR', START_TIME), WAREHOUSE_NAME
),
query_hours AS (
    SELECT
        QUERY_ID,
        DATE_TRUNC('HOUR', START_TIME) AS HOUR,
        START_TIME,
        END_TIME,
        QUERY_TYPE,
        USER_NAME,
        WAREHOUSE_NAME,
        DATABASE_NAME,
        SCHEMA_NAME,
        EXECUTION_STATUS,
        TOTAL_ELAPSED_TIME / 1000.0 / 3600.0 AS QUERY_HOURS,
        BYTES_SCANNED,
        ROWS_PRODUCED
    FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
    WHERE QUERY_TYPE NOT IN ('USE', 'SHOW', 'DESCRIBE')
        AND WAREHOUSE_NAME IS NOT NULL
),
warehouse_total_hours AS (
    SELECT
        HOUR,
        WAREHOUSE_NAME,
        SUM(QUERY_HOURS) AS TOTAL_HOURS
    FROM query_hours
    GROUP BY HOUR, WAREHOUSE_NAME
)
SELECT
    qh.QUERY_ID,
    qh.START_TIME,
    qh.END_TIME,
    qh.QUERY_TYPE,
    qh.USER_NAME,
    qh.WAREHOUSE_NAME,
    qh.DATABASE_NAME,
    qh.SCHEMA_NAME,
    qh.EXECUTION_STATUS,
    qh.QUERY_HOURS,
    qh.BYTES_SCANNED,
    qh.ROWS_PRODUCED,
    whc.TOTAL_CREDITS AS WAREHOUSE_HOURLY_CREDITS,
    wth.TOTAL_HOURS AS WAREHOUSE_HOURLY_QUERY_HOURS,
    -- Proportional credit allocation based on query execution time
    CASE 
        WHEN wth.TOTAL_HOURS > 0 THEN 
            (qh.QUERY_HOURS / wth.TOTAL_HOURS) * whc.TOTAL_CREDITS
        ELSE 0 
    END AS ESTIMATED_CREDITS,
    CASE 
        WHEN wth.TOTAL_HOURS > 0 THEN 
            (qh.QUERY_HOURS / wth.TOTAL_HOURS) * whc.TOTAL_CREDITS * $COST_PER_CREDIT
        ELSE 0 
    END AS ESTIMATED_COST_USD
FROM query_hours qh
LEFT JOIN warehouse_hourly_credits whc 
    ON qh.HOUR = whc.HOUR 
    AND qh.WAREHOUSE_NAME = whc.WAREHOUSE_NAME
LEFT JOIN warehouse_total_hours wth
    ON qh.HOUR = wth.HOUR
    AND qh.WAREHOUSE_NAME = wth.WAREHOUSE_NAME;

SELECT 'View QUERY_COSTS created successfully' AS STATUS;

## Step 3: Verify Views

Let's verify that all views were created successfully and check how much historical data is available.

In [ ]:
-- Check data availability and date ranges for each view
SELECT 'COST_WITH_CATEGORIES' AS VIEW_NAME, 
       COUNT(*) AS TOTAL_ROWS,
       MIN(USAGE_DATE) AS EARLIEST_DATE,
       MAX(USAGE_DATE) AS LATEST_DATE,
       DATEDIFF('day', MIN(USAGE_DATE), MAX(USAGE_DATE)) AS DAYS_OF_DATA
FROM COST_WITH_CATEGORIES
UNION ALL
SELECT 'WAREHOUSE_COSTS',
       COUNT(*),
       MIN(DATE(START_TIME)),
       MAX(DATE(START_TIME)),
       DATEDIFF('day', MIN(DATE(START_TIME)), MAX(DATE(START_TIME)))
FROM WAREHOUSE_COSTS
UNION ALL
SELECT 'QUERY_COSTS',
       COUNT(*),
       MIN(DATE(START_TIME)),
       MAX(DATE(START_TIME)),
       DATEDIFF('day', MIN(DATE(START_TIME)), MAX(DATE(START_TIME)))
FROM QUERY_COSTS;

In [ ]:
-- Preview recent data from each view
SELECT 'Recent Service Costs' AS SAMPLE_TYPE;
SELECT USAGE_DATE, PRODUCT_CATEGORY, SERVICE_TYPE, COST_USD
FROM COST_WITH_CATEGORIES
ORDER BY USAGE_DATE DESC
LIMIT 5;

SELECT 'Recent Warehouse Costs' AS SAMPLE_TYPE;
SELECT DATE(START_TIME) AS DATE, WAREHOUSE_NAME, COST_USD
FROM WAREHOUSE_COSTS
ORDER BY START_TIME DESC
LIMIT 5;

In [ ]:
-- Check if COST_AGENT_WH appears in the warehouse costs
-- Note: May take a few hours to appear due to ACCOUNT_USAGE data latency
SELECT 
    DATE(START_TIME) AS DATE,
    WAREHOUSE_NAME,
    SUM(COST_USD) AS TOTAL_COST
FROM WAREHOUSE_COSTS
WHERE WAREHOUSE_NAME = 'COST_AGENT_WH'
GROUP BY DATE(START_TIME), WAREHOUSE_NAME
ORDER BY DATE DESC
LIMIT 10;

-- If this returns no results, check back in a few hours after the agent has been used

## Step 4: Create Semantic Model

The semantic model tells the Cortex Analyst how to understand your data and translate natural language questions into SQL queries.

This model defines:
- Available tables and their relationships
- Dimensions (date, warehouse, category, etc.)
- Measures (costs, credits, etc.)
- Synonyms for natural language understanding
- Example queries

In [ ]:
# Create semantic model YAML for Cortex Analyst
semantic_model_yaml = '''
name: snowflake_cost_analysis
description: Analyze Snowflake costs for any time period

tables:
  - name: COST_WITH_CATEGORIES
    description: Service costs with product categories
    base_table:
      database: COST_ANALYSIS_DB
      schema: COST_VIEWS
      table: COST_WITH_CATEGORIES
    dimensions:
      - name: USAGE_DATE
        description: Date of usage
        data_type: DATE
        synonyms: [date, day, when, time, period]
      - name: SERVICE_TYPE
        description: Specific Snowflake service
        data_type: VARCHAR
        synonyms: [service, service name, type]
      - name: PRODUCT_CATEGORY
        description: Product category grouping
        data_type: VARCHAR
        synonyms: [category, product, product type]
        sample_values:
          - AI & Machine Learning
          - Data Transformation & Compute
          - Data Storage
          - Cloud Services
    measures:
      - name: CREDITS_USED
        description: Credits consumed
        data_type: NUMBER
        aggregation: SUM
      - name: COST_USD
        description: Cost in US dollars
        data_type: NUMBER
        aggregation: SUM
        synonyms: [cost, spend, spending, expense]

  - name: WAREHOUSE_COSTS
    description: Warehouse costs with compute and cloud services breakdown
    base_table:
      database: COST_ANALYSIS_DB
      schema: COST_VIEWS
      table: WAREHOUSE_COSTS
    dimensions:
      - name: START_TIME
        description: Start time of metering period
        data_type: TIMESTAMP_NTZ
        synonyms: [start, time, date, when]
      - name: WAREHOUSE_NAME
        description: Warehouse name
        data_type: VARCHAR
        synonyms: [warehouse, wh]
    measures:
      - name: COST_USD
        description: Total cost in USD
        data_type: NUMBER
        aggregation: SUM
        synonyms: [cost, spend]
      - name: COMPUTE_COST_USD
        description: Compute cost in USD
        data_type: NUMBER
        aggregation: SUM
      - name: CLOUD_SERVICES_COST_USD
        description: Cloud services cost in USD
        data_type: NUMBER
        aggregation: SUM

  - name: QUERY_COSTS
    description: Query-level costs with user and database attribution
    base_table:
      database: COST_ANALYSIS_DB
      schema: COST_VIEWS
      table: QUERY_COSTS
    dimensions:
      - name: START_TIME
        description: Query start time
        data_type: TIMESTAMP_NTZ
        synonyms: [start, time, date, when]
      - name: USER_NAME
        description: User who executed query
        data_type: VARCHAR
        synonyms: [user, username, who]
      - name: WAREHOUSE_NAME
        description: Warehouse used
        data_type: VARCHAR
        synonyms: [warehouse, wh]
    measures:
      - name: ESTIMATED_COST_USD
        description: Estimated cost in USD
        data_type: NUMBER
        aggregation: SUM
        synonyms: [cost, spend]

verified_queries:
  - name: ai_costs
    question: What is my daily AI cost for the last 30 days?
    sql: |
      SELECT USAGE_DATE, SUM(COST_USD) AS AI_COST
      FROM COST_ANALYSIS_DB.COST_VIEWS.COST_WITH_CATEGORIES
      WHERE PRODUCT_CATEGORY = 'AI & Machine Learning'
        AND USAGE_DATE >= DATEADD('day', -30, CURRENT_DATE())
      GROUP BY USAGE_DATE ORDER BY USAGE_DATE DESC
  
  - name: warehouse_costs
    question: Show me costs for a warehouse
    sql: |
      SELECT DATE(START_TIME) AS DATE, WAREHOUSE_NAME, SUM(COST_USD) AS COST
      FROM COST_ANALYSIS_DB.COST_VIEWS.WAREHOUSE_COSTS
      WHERE START_TIME >= DATEADD('day', -30, CURRENT_DATE())
      GROUP BY DATE(START_TIME), WAREHOUSE_NAME
      ORDER BY DATE DESC
  
  - name: cost_agent_costs
    question: How much does the cost agent itself cost?
    sql: |
      SELECT DATE(START_TIME) AS DATE, SUM(COST_USD) AS COST
      FROM COST_ANALYSIS_DB.COST_VIEWS.WAREHOUSE_COSTS
      WHERE WAREHOUSE_NAME = 'COST_AGENT_WH'
      GROUP BY DATE(START_TIME) ORDER BY DATE DESC
'''

print('Semantic model created successfully!')

## Step 5: Upload Semantic Model to Snowflake Stage

The semantic model needs to be stored in a Snowflake stage so the Cortex Analyst can access it.

In [ ]:
-- Create a stage to store the semantic model
CREATE STAGE IF NOT EXISTS COST_AGENT_STAGE;

SELECT 'Stage COST_AGENT_STAGE created successfully' AS STATUS;

In [ ]:
# Upload the semantic model to the stage
import tempfile
import os

# Write semantic model to a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.yaml', delete=False) as f:
    f.write(semantic_model_yaml)
    temp_file = f.name

try:
    # Upload to Snowflake stage
    put_result = session.file.put(
        temp_file,
        '@COST_AGENT_STAGE',
        auto_compress=False,
        overwrite=True
    )
    print('✅ Semantic model uploaded successfully!')
    print(put_result)
finally:
    # Clean up temporary file
    os.unlink(temp_file)

# Verify the file is in the stage
print('\nFiles in stage:')
session.sql('LIST @COST_AGENT_STAGE').show()

## Step 6: Create the Cortex Analyst Agent

**IMPORTANT**: Cortex Analyst agents must be created through the Snowflake Intelligence UI.
There is no SQL command to create them programmatically.

Follow the instructions in the next cell to create your agent.

### Instructions to Create the Cortex Analyst Agent

1. **Navigate to Snowflake Intelligence**
   - In Snowsight, look for **"AI & ML"** in the left navigation menu
   - Click on **"Cortex Analyst"** or **"Snowflake Intelligence"**

2. **Create a New Analyst**
   - Click the **"+ Analyst"** or **"Create Analyst"** button

3. **Configure Your Analyst**
   - **Name**: `Snowflake Cost Analyst`
   - **Description**: `Analyzes Snowflake costs for any time period`
   - **Warehouse**: Select `COST_AGENT_WH` from the dropdown
   - **Semantic Model**: 
     - Click **"Browse"** or **"Select File"**
     - Navigate to: `@COST_ANALYSIS_DB.COST_VIEWS.COST_AGENT_STAGE`
     - Select the YAML file that was uploaded

4. **Create the Analyst**
   - Review your configuration
   - Click **"Create"** or **"Save"**

5. **Start Using Your Agent**
   - Once created, the agent will appear in your Snowflake Intelligence interface
   - You can immediately start asking questions in natural language

### Example Questions to Try:
- What is my daily AI cost for the last 30 days?
- Show me costs for warehouse COMPUTE_WH in March
- Compare my costs from last week to this week
- What are my top 5 most expensive warehouses?
- How much does the cost agent itself cost?

**Note**: The agent will use your ACCOUNTADMIN role permissions to access the cost views.

## Step 7: Verify Your Setup

Run the queries below to verify everything was created correctly.

In [ ]:
-- Verify all components were created successfully

-- Check warehouse
SHOW WAREHOUSES LIKE 'COST_AGENT_WH';

-- Check database and schema
SHOW DATABASES LIKE 'COST_ANALYSIS_DB';
SHOW SCHEMAS LIKE 'COST_VIEWS' IN DATABASE COST_ANALYSIS_DB;

-- Check views
SHOW VIEWS IN COST_ANALYSIS_DB.COST_VIEWS;

-- Check stage and its contents
SHOW STAGES LIKE 'COST_AGENT_STAGE' IN COST_ANALYSIS_DB.COST_VIEWS;
LIST @COST_ANALYSIS_DB.COST_VIEWS.COST_AGENT_STAGE;

SELECT '✅ All components created successfully!' AS STATUS;

## Step 8: Test Your Views (Optional)

Before creating the Cortex Analyst, you can test the views directly to ensure they're working correctly.

In [ ]:
-- Test: Query costs for different time periods
SELECT 'Last 7 Days' AS PERIOD, SUM(COST_USD) AS TOTAL_COST
FROM COST_WITH_CATEGORIES
WHERE USAGE_DATE >= DATEADD('day', -7, CURRENT_DATE())
UNION ALL
SELECT 'Last 30 Days', SUM(COST_USD)
FROM COST_WITH_CATEGORIES
WHERE USAGE_DATE >= DATEADD('day', -30, CURRENT_DATE())
UNION ALL
SELECT 'Last 90 Days', SUM(COST_USD)
FROM COST_WITH_CATEGORIES
WHERE USAGE_DATE >= DATEADD('day', -90, CURRENT_DATE());

In [ ]:
-- Test: View costs by product category
SELECT 
    PRODUCT_CATEGORY,
    SUM(COST_USD) AS TOTAL_COST,
    ROUND(SUM(COST_USD) / NULLIF(SUM(SUM(COST_USD)) OVER (), 0) * 100, 2) AS PERCENT_OF_TOTAL
FROM COST_WITH_CATEGORIES
WHERE USAGE_DATE >= DATEADD('day', -30, CURRENT_DATE())
GROUP BY PRODUCT_CATEGORY
ORDER BY TOTAL_COST DESC;

## Setup Complete! 🎉

### What You've Created:
✅ **COST_AGENT_WH** - Dedicated warehouse (XSMALL, auto-suspend after 1 min)
✅ **COST_ANALYSIS_DB** - Database for cost analysis
✅ **COST_VIEWS** - Schema containing 3 cost views
✅ **COST_AGENT_STAGE** - Stage with semantic model

### Next Step:
**Create the Cortex Analyst agent** using the instructions in Step 6 above.

### Once Your Agent is Created:
Go to **Snowflake Intelligence** and start asking questions in natural language!

### Key Features:
- 📅 **Flexible time periods** - Ask about any date range (last week, March 2025, Q1, etc.)
- 📊 **Product categories** - Analyze by AI, Storage, Compute, and more
- 💰 **Cost tracking** - Monitor the agent's own costs via COST_AGENT_WH
- 🗣️ **Natural language** - No SQL required!

### Important Notes:
- **Data Latency**: ACCOUNT_USAGE views have 45 minutes to 3 hours latency
- **Historical Data**: Up to 365 days available (varies by view)
- **Cost Per Credit**: Currently set to $2.5 (update in Step 1 if different)
- **Role**: Agent uses ACCOUNTADMIN permissions

### Need Help?
- Test queries work but agent doesn't? Check the semantic model was uploaded correctly
- No data in views? Wait a few hours for ACCOUNT_USAGE to populate
- Want to track agent costs? Query WAREHOUSE_COSTS for COST_AGENT_WH

**Happy cost analyzing!** 🚀